In [54]:
import pandas as pd

df = pd.read_parquet('./taxi/fhv_tripdata_2021-01.parquet')
len(df)

1154112

In [55]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df.duration.mean()

19.1672240937939

In [56]:
df2 = df[(df.duration >= 1) & (df.duration <= 60)]
dropped = len(df)-len(df2)
print(dropped)

44286


In [58]:
import numpy as np
columns = ['PUlocationID', 'DOlocationID']
df2[columns[0]] = df2[columns[0]].replace(np.nan, -1)
df2[columns[1]] = df2[columns[1]].replace(np.nan, -1)

#percent_missing = df2[df2['PUlocationID'] == -1].sum() * 100 / len(df2)
#print(percent_missing)
#df2.head(10)

/tmp/ipykernel_3528/3376294448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[columns[0]] = df2[columns[0]].replace(np.nan, -1)
/tmp/ipykernel_3528/3376294448.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[columns[1]] = df2[columns[1]].replace(np.nan, -1)


In [34]:
df2.head(10)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000
6,B00037,2021-01-01 00:18:12,2021-01-01 00:30:04,-1.0,91.0,None,B00037,11.866667
7,B00037,2021-01-01 00:36:15,2021-01-01 00:45:08,-1.0,39.0,None,B00037,8.883333
8,B00037,2021-01-01 00:55:04,2021-01-01 01:13:02,-1.0,37.0,None,B00037,17.966667
9,B00037,2021-01-01 00:48:40,2021-01-01 01:12:02,-1.0,39.0,None,B00037,23.366667
10,B00037,2021-01-01 00:27:23,2021-01-01 00:29:25,-1.0,72.0,None,B00037,2.033333


In [63]:
def process_val_df(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df[columns[0]] = df[columns[0]].replace(np.nan, -1)
    df[columns[1]] = df[columns[1]].replace(np.nan, -1)

    return df


In [81]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df_train = process_val_df('./taxi/fhv_tripdata_2021-01.parquet')
df_val = process_val_df('./taxi/fhv_tripdata_2021-02.parquet')

train_dicts = df_train[columns].astype(str).to_dict(orient='records')
val_dicts = df_val[columns].astype(str).to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)
print(X_train.shape)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


In [ ]:
print(val_dicts[:10])
print(train_dicts[:10])

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred1 = lr.predict(X_train)
y_pred2 = lr.predict(X_val)


print(mean_squared_error(y_train, y_pred1, squared=False))
print(mean_squared_error(y_val, y_pred2, squared=False))